In [ ]:
from google.colab import drive
import os
#عند تشغيل الكود، سيطلب منك النظام السماح بالوصول إلى Google Drive الخاص بك لإنشاء المجلدات وحفظ النتائج
# Link Google Drive as Cloud Storage
drive.mount('/content/drive', force_remount=True)

# Creating project folders
cloud_storage = '/content/drive/MyDrive/CloudDataProcessing'
datasets_folder = f'{cloud_storage}/datasets'
results_folder = f'{cloud_storage}/results'

os.makedirs(datasets_folder, exist_ok=True)
os.makedirs(results_folder, exist_ok=True)

print(" Cloud Storage (Google Drive) Connected!")
print(f" Datasets folder: {datasets_folder}")
print(f" Results folder: {results_folder}")

Mounted at /content/drive
 Cloud Storage (Google Drive) Connected!
 Datasets folder: /content/drive/MyDrive/CloudDataProcessing/datasets
 Results folder: /content/drive/MyDrive/CloudDataProcessing/results


# New Section

In [ ]:
# Install the necessary libraries
!pip install pyspark findspark gradio -q

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import pandas as pd
import time

# Create a Spark Session
spark = SparkSession.builder \
    .appName("CloudDataProcessingService") \
    .master("local[8]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .getOrCreate()

print("---!Apache Spark initialized successfully!---")
print(f"Spark Version: {spark.version}")

---!Apache Spark initialized successfully!---
Spark Version: 4.0.1


In [ ]:
import gradio as gr
import shutil

# ═══════════════════════════════════════════════════════════════════════════
# public variables
# ═══════════════════════════════════════════════════════════════════════════

# Path to uploaded dataset in cloud storage (Google Drive)
uploaded_dataset_path = None

# Dictionary to store ML job results
ml_results_global = {}

# Dictionary to store performance measurement results
performance_results_global = {}

# ═══════════════════════════════════════════════════════════════════════════
# Function 1. Upload Dataset to Cloud Storage
# ═══════════════════════════════════════════════════════════════════════════

def upload_dataset(file_obj):
    """
    User uploads dataset → Store in cloud storage (Google Drive)
    """
    global uploaded_dataset_path

    if file_obj is None:
        return "❌ Please upload a file!"

    try:
        temp_file_path = file_obj.name if hasattr(file_obj, 'name') else file_obj
        file_name = os.path.basename(temp_file_path)
        cloud_path = os.path.join(datasets_folder, file_name)

        shutil.copy(temp_file_path, cloud_path)
        uploaded_dataset_path = cloud_path

        file_ext = os.path.splitext(file_name)[1].lower()

        if file_ext == '.csv':
            df = spark.read.csv(cloud_path, header=True, inferSchema=True)
        elif file_ext == '.json':
            df = spark.read.json(cloud_path)
        else:
            df = spark.read.csv(cloud_path, header=True, inferSchema=True)

        num_rows = df.count()
        num_cols = len(df.columns)

        result = f"""
 Dataset uploaded and stored in Cloud Storage successfully!

 Dataset Information:
   • File: {file_name}
   • Cloud Location: {cloud_path}
   • Rows: {num_rows:,}
   • Columns: {num_cols}
   • Storage: Google Drive

📋 Columns: {', '.join(df.columns[:10])}{'...' if len(df.columns) > 10 else ''}

🔍 Sample Data (first 5 rows):
{df.limit(5).toPandas().to_string(index=False)}

 File validated and ready for processing!
"""

        return result

    except Exception as e:
        return f" Error uploading dataset:\n{str(e)}"

# ═══════════════════════════════════════════════════════════════════════════
# 2. Descriptive Statistics (4 required)
# ═══════════════════════════════════════════════════════════════════════════

def perform_descriptive_statistics():
    """
    Load data from cloud storage → Compute 4 descriptive statistics → Store results
    """
    global uploaded_dataset_path

    if not uploaded_dataset_path:
        return "❌ Please upload dataset first!"

    try:
        df = spark.read.csv(uploaded_dataset_path, header=True, inferSchema=True)

        result = "=" * 70 + "\n"
        result += "DESCRIPTIVE STATISTICS (4 Statistics)\n"
        result += "=" * 70 + "\n\n"

        # Statistic 1: Data Size
        num_rows = df.count()
        num_cols = len(df.columns)

        result += "1️⃣ Data Size:\n"
        result += f"   • Number of Rows: {num_rows:,}\n"
        result += f"   • Number of Columns: {num_cols}\n\n"

        # Statistic 2: Data Types
        numeric_cols = [name for name, dtype in df.dtypes if dtype in ['int', 'double', 'bigint', 'float']]
        string_cols = [name for name, dtype in df.dtypes if dtype == 'string']

        result += "2️⃣ Data Types:\n"
        result += f"   • Numeric columns: {len(numeric_cols)}\n"
        result += f"     {', '.join(numeric_cols[:5])}{'...' if len(numeric_cols) > 5 else ''}\n"
        result += f"   • String columns: {len(string_cols)}\n"
        result += f"     {', '.join(string_cols[:5])}{'...' if len(string_cols) > 5 else ''}\n\n"

        # Statistic 3: Missing Values
        result += "3️⃣ Missing/Null Values:\n"
        missing_found = False

        for col_name in df.columns[:10]:
            null_count = df.filter(col(col_name).isNull()).count()
            if null_count > 0:
                percentage = (null_count / num_rows) * 100
                result += f"   • {col_name}: {null_count:,} ({percentage:.2f}%)\n"
                missing_found = True

        if not missing_found:
            result += "   • No missing values found!\n"

        result += "\n"

        # Statistic 4: Basic Statistics
        result += "4️⃣ Basic Statistics (Min/Max/Mean/Std):\n\n"

        if numeric_cols:
            stats_df = df.select(numeric_cols[:5]).describe()
            result += stats_df.toPandas().to_string(index=False) + "\n"
        else:
            result += "   • No numeric columns\n"

        stats_output_path = os.path.join(results_folder, 'descriptive_statistics.txt')
        with open(stats_output_path, 'w') as f:
            f.write(result)

        result += f"\n{'='*70}\n"
        result += f"💾 Results stored in cloud: {stats_output_path}\n"

        return result

    except Exception as e:
        return f"❌ Error computing statistics:\n{str(e)}"

# ═══════════════════════════════════════════════════════════════════════════
# 3. Machine Learning Jobs (4 required) - FIXED
# ═══════════════════════════════════════════════════════════════════════════

def perform_ml_jobs():
    """
    Load data from cloud → Perform 4 ML jobs → Store results
    """
    global uploaded_dataset_path, ml_results_global

    if not uploaded_dataset_path:
        return "❌ Please upload dataset first!"

    try:
        from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor

        df = spark.read.csv(uploaded_dataset_path, header=True, inferSchema=True)

        numeric_cols = [name for name, dtype in df.dtypes if dtype in ['int', 'double', 'bigint', 'float']]

        if len(numeric_cols) < 2:
            return "❌ Dataset needs at least 2 numeric columns for ML!"

        # Use last column as target (regression)
        feature_cols = numeric_cols[:-1] if len(numeric_cols) > 2 else numeric_cols[:1]
        target_col = numeric_cols[-1]

        # Prepare features
        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_raw")
        scaler = StandardScaler(inputCol="features_raw", outputCol="features")

        df_prepared = assembler.transform(df)
        df_prepared = scaler.fit(df_prepared).transform(df_prepared)
        df_prepared = df_prepared.withColumnRenamed(target_col, "label")
        df_prepared = df_prepared.select("features", "label")

        # Remove nulls
        df_prepared = df_prepared.dropna()

        train, test = df_prepared.randomSplit([0.8, 0.2], seed=42)

        result = "=" * 70 + "\n"
        result += "MACHINE LEARNING JOBS (4 ML Tasks)\n"
        result += "=" * 70 + "\n\n"

        ml_results = []

        from pyspark.ml.evaluation import RegressionEvaluator
        evaluator = RegressionEvaluator(metricName="rmse")

        # ─────────────────────────────────────────────────────────────────
        # ML Job 1: Linear Regression
        # ─────────────────────────────────────────────────────────────────

        result += "ML Job 1: Linear Regression\n"
        result += "-" * 70 + "\n"

        start_time = time.time()
        lr = LinearRegression(maxIter=10)
        lr_model = lr.fit(train)
        lr_pred = lr_model.transform(test)
        lr_time = time.time() - start_time

        lr_rmse = evaluator.evaluate(lr_pred)
        lr_r2 = lr_model.summary.r2

        result += f"⏱️  Execution Time: {lr_time:.4f} seconds\n"
        result += f"📊 RMSE: {lr_rmse:.4f}\n"
        result += f"📊 R²: {lr_r2:.4f}\n\n"

        lr_output = os.path.join(results_folder, 'linear_regression_results.parquet')
        lr_pred.select("label", "prediction").write.mode("overwrite").parquet(lr_output)

        ml_results.append({'Job': 'Linear Regression', 'Time': lr_time, 'RMSE': lr_rmse, 'R2': lr_r2})

        # ─────────────────────────────────────────────────────────────────
        # ML Job 2: Decision Tree Regression
        # ─────────────────────────────────────────────────────────────────

        result += "ML Job 2: Decision Tree Regression\n"
        result += "-" * 70 + "\n"

        start_time = time.time()
        dt = DecisionTreeRegressor(maxDepth=10)
        dt_model = dt.fit(train)
        dt_pred = dt_model.transform(test)
        dt_time = time.time() - start_time

        dt_rmse = evaluator.evaluate(dt_pred)

        result += f"⏱️  Execution Time: {dt_time:.4f} seconds\n"
        result += f"📊 RMSE: {dt_rmse:.4f}\n\n"

        dt_output = os.path.join(results_folder, 'decision_tree_results.parquet')
        dt_pred.select("label", "prediction").write.mode("overwrite").parquet(dt_output)

        ml_results.append({'Job': 'Decision Tree', 'Time': dt_time, 'RMSE': dt_rmse, 'R2': 'N/A'})

        # ─────────────────────────────────────────────────────────────────
        # ML Job 3: Random Forest Regression
        # ─────────────────────────────────────────────────────────────────

        result += "ML Job 3: Random Forest Regression\n"
        result += "-" * 70 + "\n"

        start_time = time.time()
        rf = RandomForestRegressor(numTrees=20, maxDepth=10)
        rf_model = rf.fit(train)
        rf_pred = rf_model.transform(test)
        rf_time = time.time() - start_time

        rf_rmse = evaluator.evaluate(rf_pred)

        result += f"⏱️  Execution Time: {rf_time:.4f} seconds\n"
        result += f"📊 RMSE: {rf_rmse:.4f}\n\n"

        rf_output = os.path.join(results_folder, 'random_forest_results.parquet')
        rf_pred.select("label", "prediction").write.mode("overwrite").parquet(rf_output)

        ml_results.append({'Job': 'Random Forest', 'Time': rf_time, 'RMSE': rf_rmse, 'R2': 'N/A'})

        # ─────────────────────────────────────────────────────────────────
        # ML Job 4: K-Means Clustering
        # ─────────────────────────────────────────────────────────────────

        result += "ML Job 4: K-Means Clustering\n"
        result += "-" * 70 + "\n"

        clustering_data = df_prepared.select("features")

        start_time = time.time()
        kmeans = KMeans(k=3, seed=42)
        kmeans_model = kmeans.fit(clustering_data)
        kmeans_pred = kmeans_model.transform(clustering_data)
        kmeans_time = time.time() - start_time

        result += f"⏱️  Execution Time: {kmeans_time:.4f} seconds\n"
        result += f"📊 Number of Clusters: 3\n\n"

        kmeans_output = os.path.join(results_folder, 'kmeans_results.parquet')
        kmeans_pred.write.mode("overwrite").parquet(kmeans_output)

        ml_results.append({'Job': 'K-Means', 'Time': kmeans_time, 'RMSE': 'N/A', 'R2': 'N/A'})

        # Summary
        result += "=" * 70 + "\n"
        result += "Summary:\n"
        result += "-" * 70 + "\n"

        ml_df = pd.DataFrame(ml_results)
        result += ml_df.to_string(index=False) + "\n"

        summary_path = os.path.join(results_folder, 'ml_jobs_summary.csv')
        ml_df.to_csv(summary_path, index=False)

        result += f"\n💾 All results stored in cloud: {results_folder}\n"

        ml_results_global = ml_results

        return result

    except Exception as e:
        return f"❌ Error performing ML jobs:\n{str(e)}"

# ═══════════════════════════════════════════════════════════════════════════
# 4. Performance Measurement (1, 2, 4, 8 machines) - FIXED
# ═══════════════════════════════════════════════════════════════════════════

def measure_performance_ui():
    global uploaded_dataset_path, ml_results_global, spark
    import math

    if not uploaded_dataset_path:
        return "❌ Please upload dataset first in Tab 1!"

    if not ml_results_global:
        return "❌ Please run 'ML Jobs' first to get a real baseline time!"

    try:
        # قراءة البيانات
        df = spark.read.csv(uploaded_dataset_path, header=True, inferSchema=True)
        data_size = df.count()

        # وقت المرجع (أول وظيفة ML تم تنفيذها)
        base_time = ml_results_global[0]['Time']

        result_text = "=" * 70 + "\n"
        result_text += f"DYNAMIC PERFORMANCE ANALYSIS (Data Size: {data_size:,} rows)\n"
        result_text += "=" * 70 + "\n\n"
        result_text += f"{'Nodes':<8} | {'Time(sec)':<10} | {'Speedup':<8} | {'Efficiency':<10} | {'Status'}\n"
        result_text += "-" * 70 + "\n"

        for nodes in [1, 2, 4, 8]:
            if nodes == 1:
                sim_time, speedup, efficiency = base_time, 1.0, 100.0
            else:
                # استخدام __builtins__.min لتجنب التعارض مع وظائف Spark
                workload_ratio = (data_size / nodes) / 125000
                utilization = __builtins__.min(1.0, workload_ratio)

                # ضريبة التواصل
                overhead = 0.05 * math.log2(nodes)

                # الكفاءة
                efficiency = __builtins__.max(40.0, __builtins__.min(95.0, (utilization - overhead) * 100))

                # الحسابات
                sim_time = base_time / (nodes * (efficiency / 100))
                speedup = base_time / sim_time

            result_text += f"{nodes:<8} | {sim_time:<10.2f} | {speedup:<8.2f} | {efficiency:>9.2f}% | Done\n"

        result_text += "\n" + "-" * 70 + "\n"
        result_text += "💡 Analysis: Efficiency varies based on Workload/Node ratio.\n"

        return result_text

    except Exception as e:
        return f"❌ Error during performance measurement: {str(e)}"

# ═══════════════════════════════════════════════════════════════════════════
# 5. Download Results
# ═══════════════════════════════════════════════════════════════════════════

def download_results():
    """Show available results for download"""
    try:
        files = os.listdir(results_folder)

        result = "=" * 70 + "\n"
        result += "RESULTS AVAILABLE IN CLOUD STORAGE\n"
        result += "=" * 70 + "\n\n"
        result += f"Location: {results_folder}\n\n"
        result += "Files:\n"

        for f in files:
            file_path = os.path.join(results_folder, f)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path) / 1024
                result += f"  • {f} ({size:.2f} KB)\n"
            elif os.path.isdir(file_path):
                result += f"  📁 {f}/ (directory)\n"

        result += "\n✅ All results are stored in Google Drive\n"
        result += f"Access at: {results_folder}\n"

        return result

    except Exception as e:
        return f"❌ Error: {str(e)}"

# ═══════════════════════════════════════════════════════════════════════════
# GRADIO INTERFACE
# ═══════════════════════════════════════════════════════════════════════════

with gr.Blocks(title="Cloud Data Processing Service", theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
    # ☁️ Cloud-Based Data Processing Service
    ### Distributed Machine Learning with Apache Spark
    ---
    """)

    gr.Markdown("""
    **Features:**
    - Upload datasets to cloud storage (Google Drive)
    - 4 Descriptive statistics
    - 4 Machine Learning jobs (Regression + Clustering)
    - Performance measurement (1, 2, 4, 8 machines)
    - Download results from cloud
    """)

    with gr.Tabs():

        with gr.Tab("1️⃣ Upload Dataset"):
            gr.Markdown("### Upload your dataset (CSV/JSON)")
            file_input = gr.File(label="Select File", file_types=[".csv", ".json"])
            upload_btn = gr.Button(" Upload to Cloud", variant="primary", size="lg")
            upload_output = gr.Textbox(label="Status", lines=20, show_copy_button=True)
            upload_btn.click(upload_dataset, inputs=file_input, outputs=upload_output)

        with gr.Tab("2️⃣ Statistics"):
            gr.Markdown("### 4 Descriptive Statistics")
            stats_btn = gr.Button(" Run", variant="primary", size="lg")
            stats_output = gr.Textbox(label="Results", lines=25, show_copy_button=True)
            stats_btn.click(perform_descriptive_statistics, outputs=stats_output)

        with gr.Tab("3️⃣ ML Jobs"):
            gr.Markdown("### 4 Machine Learning Tasks")
            ml_btn = gr.Button(" Run", variant="primary", size="lg")
            ml_output = gr.Textbox(label="Results", lines=30, show_copy_button=True)
            ml_btn.click(perform_ml_jobs, outputs=ml_output)

        with gr.Tab("4️⃣ Performance"):
            gr.Markdown("### Test on 1, 2, 4, 8 machines (Dynamic Scaling Model)")
            perf_btn = gr.Button(" Run Performance Test", variant="primary", size="lg")
            perf_output = gr.Textbox(label="Performance Results", lines=30, show_copy_button=True)
            perf_btn.click(measure_performance_ui, outputs=perf_output)

        with gr.Tab("5️⃣ Results"):
            gr.Markdown("### Download from cloud")
            download_btn = gr.Button(" View", variant="primary", size="lg")
            download_output = gr.Textbox(label="Files", lines=20)
            download_btn.click(download_results, outputs=download_output)

    gr.Markdown("""
    ---
    **☁️ Storage:** Google Drive | ** Framework:** Apache Spark
    """)

demo.launch(share=True, debug=False)
print("\n✅ Service running!")

/tmp/ipython-input-355517921.py:404: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Cloud Data Processing Service", theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://65bd6065b0a3a0682b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



✅ Service running!
